# Classify and Group Punts by Injury Cause/Type

There are 37 Punt plays that resulted in concussions. Not all of them were the same type of injury cause. We will try to classify them as the "type" of injury play.

- 

In [18]:
import pandas as pd
pd.set_option('display.max_columns', 50)


In [19]:
# Read in non-NGS data sources
ppd = pd.read_csv('../input/player_punt_data.csv')
gd = pd.read_csv('../input/game_data.csv')
pprd = pd.read_csv('../input/play_player_role_data.csv')
vr = pd.read_csv('../input/video_review.csv')
vfi = pd.read_csv('../input/video_footage-injury.csv')
pi = pd.read_csv('../input/play_information.csv')

all_dfs = [ppd, gd, pprd, vr, vfi, pi]
for mydf in all_dfs:
    mydf.columns = [col.lower() for col in mydf.columns]
    
ri = pd.read_csv('../working/role_info.csv')

In [20]:
vr_merged = pd.merge(vr, pprd)
vr_merged = pd.merge(vr_merged, ri)

vr_merged = pd.merge(vr_merged, pprd, left_on=['season_year','gamekey','playid','primary_partner_gsisid'],
                     right_on=['season_year','gamekey','playid', 'gsisid'], how='left',
                     suffixes=('', '_primary_partner'))
vr_merged = pd.merge(vr_merged, ri, left_on='role_primary_partner', right_on='role', how='left', suffixes=('','_primary_partner'))

vr_merged = vr_merged.fillna('None')
vr_merged['count'] = 1

In [21]:
vr_merged = pd.merge(vr_merged, pi)

In [29]:
vr_merged.columns

Index(['season_year', 'gamekey', 'playid', 'gsisid', 'player_activity_derived',
       'turnover_related', 'primary_impact_type', 'primary_partner_gsisid',
       'primary_partner_activity_derived', 'friendly_fire', 'role',
       'Unnamed: 0', 'generalized_role', 'punting_returning_team',
       'gsisid_primary_partner', 'role_primary_partner',
       'Unnamed: 0_primary_partner', 'role_primary_partner',
       'generalized_role_primary_partner',
       'punting_returning_team_primary_partner', 'count', 'season_type',
       'game_date', 'week', 'game_clock', 'yardline', 'quarter', 'play_type',
       'poss_team', 'home_team_visit_team', 'score_home_visiting',
       'playdescription'],
      dtype='object')

In [72]:
rename_player_activity = {'Blocked' : 1,
                          'Blocking' : 2,
                          'Tackled' : 3,
                          'Tackling' : 4,
                          'Unclear' : 5,
                          'None' : 6}

role_rename = {'Punt_Returner' : 1, 'Defensive_Lineman' : 2, 'None' : 3, 'Defensive_Backer' : 4,
       'Gunner' : 5, 'Punting_Lineman' : 6, 'Punter_Protector' : 7, 'Punting_Wing' : 8,
       'Jammer' : 9, 'Punting_Longsnapper': 10, 'PuntFullBack' : 11 , 'Punter' : 12}

impact_rename = {'Helmet-to-body' : 1,
                 'Helmet-to-ground' : 2,
                 'Helmet-to-helmet' : 3,
                 'Unclear' : 4}

friendly_fire_rename = {'Yes' :  1, 'No' : 2, 'Unclear' : 3}

punting_returning_rename = {'Punting_Team' : 1, 'Returning_Team' : 2}

In [74]:
vr_for_clustering = vr_merged[['primary_impact_type', 'primary_partner_activity_derived', 'friendly_fire', 'generalized_role',
          'generalized_role_primary_partner', 'punting_returning_team','punting_returning_team_primary_partner']] \
    .replace(impact_rename) \
    .replace(rename_player_activity) \
    .replace(friendly_fire_rename) \
    .replace(punting_returning_rename) \
    .replace(role_rename)

In [86]:
import numpy as np
from kmodes.kmodes import KModes

# random categorical data
data = np.random.choice(20, (100, 10))

km = KModes(n_clusters=5, init='Huang', n_init=20, verbose=1)

clusters = km.fit_predict(vr_for_clustering)

# Print the cluster centroids
print(km.cluster_centroids_)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 11, cost: 58.0
Run 1, iteration: 2/100, moves: 2, cost: 58.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 6, cost: 80.0
Run 2, iteration: 2/100, moves: 6, cost: 78.0
Run 2, iteration: 3/100, moves: 0, cost: 78.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 8, cost: 74.0
Run 3, iteration: 2/100, moves: 4, cost: 72.0
Run 3, iteration: 3/100, moves: 7, cost: 66.0
Run 3, iteration: 4/100, moves: 3, cost: 66.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 3, cost: 72.0
Run 4, iteration: 2/100, moves: 0, cost: 72.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 5, cost: 72.0
Run 5, iteration: 2/100, moves: 3, cost: 72.0
I

In [87]:
vr_merged['cluster'] = clusters

In [89]:
vr_merged

,season_year,gamekey,playid,gsisid,player_activity_derived,turnover_related,primary_impact_type,primary_partner_gsisid,primary_partner_activity_derived,friendly_fire,role,Unnamed: 0,generalized_role,punting_returning_team,gsisid_primary_partner,role_primary_partner,Unnamed: 0_primary_partner,role_primary_partner,generalized_role_primary_partner,punting_returning_team_primary_partner,count,season_type,game_date,week,game_clock,yardline,quarter,play_type,poss_team,home_team_visit_team,score_home_visiting,playdescription,cluster
0,2016,5,3129,31057,Tackling,No,Helmet-to-body,32482,Tackled,No,PLW,26,Punting_Wing,Punting_Team,32482,PR,28,PR,Punt_Returner,Returning_Team,1,Pre,08/11/2016,2,03:44,CHI 31,3,Punt,CHI,CHI-DEN,0 - 22,(3:44) (Punt formation) P.O'Donnell punts 58 y...,1
1,2016,280,2918,32120,Blocking,No,Helmet-to-body,32725,Blocked,No,PLW,26,Punting_Wing,Punting_Team,32725,PDR1,14,PDR1,Defensive_Lineman,Returning_Team,1,Reg,12/18/2016,15,03:36,JAX 21,3,Punt,JAX,HST-JAX,8 - 20,"(3:36) B.Nortman punts 49 yards to HST 30, Cen...",1
2,2017,414,1262,33941,Tackling,No,Helmet-to-ground,None,None,None,PLW,26,Punting_Wing,Punting_Team,None,None,None,None,None,None,1,Reg,09/11/2017,1,10:20,LAC 28,2,Punt,LAC,DEN-LAC,7 - 7,"(10:20) D.Kaser punts 59 yards to DEN 13, Cent...",3
3,2017,607,978,29793,Blocking,No,Helmet-to-helmet,32114,Blocked,No,PLW,26,Punting_Wing,Punting_Team,32114,PDR2,15,PDR2,Defensive_Lineman,Returning_Team,1,Reg,12/14/2017,15,14:11,DEN 38,2,Punt,IND,IND-DEN,7 - 0,(14:11) (Punt formation) R.Sanchez punts 27 ya...,4
4,2016,21,2587,29343,Blocked,No,Helmet-to-helmet,31059,Blocking,No,GL,0,Gunner,Punting_Team,31059,PLL1,22,PLL1,Defensive_Backer,Returning_Team,1,Pre,08/20/2016,3,05:52,TEN 45,3,Punt,CAR,TEN-CAR,10 - 13,(5:52) (Punt formation) K.Redfern punts 36 yar...,4
5,2016,280,3746,27654,Tackling,No,Helmet-to-helmet,33127,Tackled,No,GL,0,Gunner,Punting_Team,33127,PR,28,PR,Punt_Returner,Returning_Team,1,Reg,12/18/2016,15,05:57,JAX 15,4,Punt,JAX,HST-JAX,14 - 20,"(5:57) B.Nortman punts 41 yards to HST 44, Cen...",1
6,2016,289,2341,32007,Blocked,No,Helmet-to-helmet,32998,Blocking,No,GL,0,Gunner,Punting_Team,32998,PDR3,16,PDR3,Defensive_Lineman,Returning_Team,1,Reg,12/19/2016,15,12:41,WAS 26,3,Punt,WAS,WAS-CAR,9 - 20,"(12:41) T.Way punts 51 yards to CAR 23, Center...",4
7,2016,296,2667,32783,Tackling,No,Helmet-to-helmet,32810,Tackling,Yes,GL,0,Gunner,Punting_Team,32810,GR,3,GR,Gunner,Punting_Team,1,Reg,12/24/2016,16,06:11,TEN 34,3,Punt,TEN,JAX-TEN,22 - 10,"(6:11) B.Kern punts 43 yards to JAX 23, Center...",0
8,2016,29,538,31023,Tackling,No,Helmet-to-body,31941,Tackled,No,GR,3,Gunner,Punting_Team,31941,PR,28,PR,Punt_Returner,Returning_Team,1,Pre,08/19/2016,3,04:46,NYJ 22,1,Punt,NYJ,WAS-NYJ,0 - 0,"(4:46) L.Edwards punts 51 yards to WAS 27, Cen...",1
9,2016,45,1212,33121,Tackling,No,Helmet-to-body,28249,Tackled,No,PRT,30,Punting_Lineman,Punting_Team,28249,PR,28,PR,Punt_Returner,Returning_Team,1,Pre,08/27/2016,4,08:29,NYG 46,2,Punt,NYG,NYJ-NYG,0 - 0,"(8:29) B.Wing punts 44 yards to NYJ 10, Center...",1
